## Environnement

### Installation des dépendances

In [ ]:
%pip install pandas
%pip install requests
%pip install tqdm
%pip install ipywidgets

### Import des librairies

In [2]:
import os
import requests
import pandas as pd
import csv
import re
from contextlib import redirect_stderr
import io

## Fonctions

### Suppression des espaces qui se sont glissés dans l'export

*Les adresses des sites des bibliothèques spécialisées de Paris comportaient un espace entre la fin de l'adresse et le ".fr" empêchant l'automatisation*

Codé avec Github Copilot

In [ ]:
# Chemin du fichier CSV
#input_file = '/home/mathieu/Documents/Stage/datasets/dataset_ExpoUniv1937/rapport_33396.csv'
#output_file = '/home/mathieu/Documents/Stage/datasets/dataset_ExpoUniv1937/rapport_33396_cleaned_true.csv'

# Fonction pour nettoyer les URL
def clean_url(url):
    return url.replace("paris .fr", "paris.fr")

# Lecture et écriture du fichier CSV
with open(input_file, mode='r', encoding='utf-8') as infile, \
     open(output_file, mode='w', encoding='utf-8', newline='') as outfile:
    
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    # Traiter chaque ligne
    for row in reader:
        # Nettoyer chaque colonne de la ligne
        cleaned_row = [clean_url(cell) if "https://bibliotheques-specialisees.paris" in cell else cell for cell in row]
        writer.writerow(cleaned_row)

print(f"Le fichier nettoyé a été enregistré sous : {output_file}")

### Supression des coats

Codé avec Github Copilot

In [ ]:
#input_file = "/home/mathieu/Documents/Stage/datasets/dataset_ExpoUniv1937/rapport_33396_cleaned_true.csv"

# Lire le contenu du fichier
with open(input_file, "r", encoding="utf-8") as file:
    content = file.read()

# Supprimer tous les guillemets
content = re.sub(r'"""', '', content)
content = re.sub(r'""', '', content)
content = re.sub(r'"', '', content)

# Écrire le contenu modifié dans le fichier
with open(input_file, "w", encoding="utf-8") as file:
    file.write(content)

print("Les guillemets ont été supprimés avec succès.")

In [64]:
input_csv = "/home/mathieu/Documents/Stage/datasets/dataset_ExpoUniv1937/test.csv"

### Détecter les lignes fautives d'un csv

+ Utilisation de la librairie io et de redirect_stderr pour capturer les lignes fautives du csv ne comportant pas le bon nombre de colonnes

*Le besoin s'est fait ressentir de passer par une phase préalable de troubleshooting face à l'impossibilité de lire le csv avec un simple pd.read_csv*

Excepté une petite modification dû à un changement dans la syntaxe de pandas dans la version 1.3.0 (concernant le paramètre "on_bad_lines") les credits vont à :

© vlogize https://www.youtube.com/watch?v=1fdePybJ0yQ
CC BY-SA 4.0

Based on a Stack Overflow exchange https://stackoverflow.com/users/6238676/prish
Also under CC BY-SA 4.0

In [ ]:
# Prepare capture warnings
f = io.StringIO()
# warning_str = ""

# Read the csv while capturing warnings
with redirect_stderr(f):
    df = pd.read_csv(input_csv, sep=';', encoding="utf-8", on_bad_lines='warn')

# Extract warning messages
if f.getvalue():
    warning_str = f.getvalue()

# Using Regex to capture line numbers
regex = re.compile('line ([0-9]*)')
line_numbers = regex.findall(warning_str)

# Output line numbers
print(line_numbers)


[]


*Finalement l'avoir ouvert avec libreoffice et enregistré après a réglé le problème en réécrivant automatiquement un nombre cohérent de colonnes*

l'erreur de base : pandas ParserError: Error tokenizing data. C error: Expected 16 fields in line 14, saw 17

In [ ]:
print("Contenu capturé de stderr :")
print(warning_str)

In [14]:
test_csv = "/home/mathieu/Documents/Stage/datasets/dataset_ExpoUniv1937/test.csv"

In [15]:
df = pd.read_csv(test_csv, sep=';', encoding='utf-8')

### Ajouter une colonne pour le chemin des fichiers dans le csv

In [68]:
#df.loc[1]
df.head()

,URL,TITRE,TYPE,AUTEURS,CONTRIBUTEURS,EDITEURS,DATES,DESCRIPTIONS,SUJETS,PROVENANCE,ARK CATALOGUE,DROITS,NOMBRE DE VUES,CODE(S) PROJET(S),NQA Moyen (%),SOURCE,FICHIER
0,https://bibliotheques-specialisees.paris.fr/ar...,[Etude publicitaire pour le] cognac Bisquit. [...,images,"Vigneau, André (1892-1968) ¤ Camera (Société d...",NaN,1937,1937,Caméra fut un studio de publicité photographiq...,Cognac Bisquit ¤ Exposition internationale . 1...,Bibliothèques spécialisées de la Ville de Paris,NaN,domaine public ¤ public domain,1.0,NaN,0.0,© André Vigneau / BHVP / Roger-Viollet / Ville...,/home/mathieu/Documents/Stage/datasets/dataset...
1,https://bibliotheques-specialisees.paris.fr/ar...,[Etude publicitaire pour Robert Lang. Expositi...,images,"Vigneau, André (1892-1968) ¤ Camera (Société d...",NaN,[1937 ],1937,Caméra fut un studio de publicité photographiq...,Exposition internationale . 1937 . Paris ¤ Pho...,Bibliothèques spécialisées de la Ville de Paris,NaN,domaine public ¤ public domain,1.0,NaN,0.0,© André Vigneau / BHVP / Roger-Viollet / Ville...,/home/mathieu/Documents/Stage/datasets/dataset...
2,https://gallica.bnf.fr/ark:/12148/btv1b10589915p,[Exposition des Arts et des Techniques à Paris...,images,"Maillard, Marcel (1899-1977). Photographe. Anc...",NaN,NaN,1937,"Donateur : Manneville, Philippe (1937 -2012). ...",Expositions internationales ¤ Paris (France),Gallica,http://catalogue.bnf.fr/ark:/12148/cb45893386h,domaine public ¤ public domain,1.0,BmLHav000,0.0,"Bibliothèque municipale du Havre, MM/FP 64.02",/home/mathieu/Documents/Stage/datasets/dataset...
3,https://gallica.bnf.fr/ark:/12148/btv1b105899181,[Exposition des Arts et des Techniques à Paris...,images,"Maillard, Marcel (1899-1977). Photographe. Anc...",NaN,NaN,1937,"Donateur : Manneville, Philippe (1937 -2012). ...",Expositions internationales ¤ Paris (France),Gallica,http://catalogue.bnf.fr/ark:/12148/cb45893389j,domaine public ¤ public domain,1.0,BmLHav000,0.0,"Bibliothèque municipale du Havre, MM/FP 64.05",/home/mathieu/Documents/Stage/datasets/dataset...
4,https://gallica.bnf.fr/ark:/12148/btv1b10589913s,[Exposition des Arts et des Techniques à Paris...,images,"Maillard, Marcel (1899-1977). Photographe. Anc...",NaN,NaN,1937,"Donateur : Manneville, Philippe (1937 -2012). ...",Expositions internationales ¤ Paris (France),Gallica,http://catalogue.bnf.fr/ark:/12148/cb45893384t,domaine public ¤ public domain,1.0,BmLHav000,0.0,"Bibliothèque municipale du Havre, MM/FP 63.11",/home/mathieu/Documents/Stage/datasets/dataset...


### Afficher les colonnes

In [17]:
df.columns

Index(['URL', 'TITRE', 'TYPE', 'AUTEURS', 'CONTRIBUTEURS', 'EDITEURS', 'DATES',
       'DESCRIPTIONS', 'SUJETS', 'PROVENANCE', 'ARK CATALOGUE', 'DROITS',
       'NOMBRE DE VUES', 'CODE(S) PROJET(S)', 'NQA Moyen (%)', 'SOURCE'],
      dtype='object')

### Ajouter une colonne "FICHIER" et la peupler avec les emplacements des fichiers

In [ ]:
# df.insert(16, 'FICHIER', [
#     f"/home/mathieu/Documents/Stage/datasets/dataset_ExpoUniv1937/output/image_{i}.jpg"
#     for i in range(1, len(df) + 1)
# ])
# print(df)

                                                    URL  \
0     https://bibliotheques-specialisees.paris.fr/ar...   
1     https://bibliotheques-specialisees.paris.fr/ar...   
2      https://gallica.bnf.fr/ark:/12148/btv1b10589915p   
3      https://gallica.bnf.fr/ark:/12148/btv1b105899181   
4      https://gallica.bnf.fr/ark:/12148/btv1b10589913s   
...                                                 ...   
1235  https://bibliotheques-specialisees.paris.fr/ar...   
1236  https://bibliotheques-specialisees.paris.fr/ar...   
1237  https://bibliotheques-specialisees.paris.fr/ar...   
1238  https://bibliotheques-specialisees.paris.fr/ar...   
1239    https://gallica.bnf.fr/ark:/12148/btv1b90104852   

                                                  TITRE    TYPE  \
0     [Etude publicitaire pour le] cognac Bisquit. [...  images   
1     [Etude publicitaire pour Robert Lang. Expositi...  images   
2     [Exposition des Arts et des Techniques à Paris...  images   
3     [Exposition des A

### Changer le nom des colonnes


In [ ]:
# df.columns = ['INDEX: 0', 'URL', 'TITRE', 'TYPE', 'AUTEURS', 'CONTRIBUTEURS',
#        'EDITEURS', 'DATES', 'DESCRIPTIONS', 'SUJETS', 'PROVENANCE',
#        'ARK CATALOGUE', 'DROITS', 'NOMBRE DE VUES', 'CODE(S) PROJET(S)',
#        'NQA Moyen (%)', 'SOURCE', 'FICHIER']

In [19]:
df.columns

Index(['URL', 'TITRE', 'TYPE', 'AUTEURS', 'CONTRIBUTEURS', 'EDITEURS', 'DATES',
       'DESCRIPTIONS', 'SUJETS', 'PROVENANCE', 'ARK CATALOGUE', 'DROITS',
       'NOMBRE DE VUES', 'CODE(S) PROJET(S)', 'NQA Moyen (%)', 'SOURCE',
       'FICHIER'],
      dtype='object')

In [20]:
df.to_csv(test_csv)

## Processing


In [61]:
test_url_bib = 'https://bibliotheques-specialisees.paris.fr/ark:/73873/pf0001836977'
test_url_gallica = 'https://gallica.bnf.fr/ark:/12148/btv1b10589915p'

In [62]:
resultat = requests.get(test_url_gallica)
if resultat.status_code != 200:
    print('Ah, erreur !')
resultat

<Response [200]>

In [63]:
resultat.headers

{'Date': 'Tue, 17 Jun 2025 12:41:26 GMT', 'Server': 'Apache', 'Set-Cookie': 'dtCookie=v_4_srv_1_sn_DD06A6FF8243424F7A43D698583602D6_perc_100000_ol_0_mul_1_app-3A3c476fda10179998_0; Path=/; Domain=.bnf.fr, JSESSIONID=061017191FA44603FE804F6A92AAC234; Path=/; Secure; HttpOnly', 'X-OneAgent-JS-Injection': 'true', 'Server-Timing': 'dtRpid;desc="-2023569636", dtSInfo;desc="0"', 'Content-Type': 'text/html;charset=UTF-8', 'Content-Language': 'fr-FR', 'Vary': 'Accept-Encoding,User-Agent', 'Content-Encoding': 'gzip', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; always', 'Keep-Alive': 'timeout=5, max=500', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}

## Correction intitulé des fichiers

In [4]:
import pandas as pd

# Chargement de ton CSV
df = pd.read_csv("/home/mathieu/Documents/Stage/datasets/dataset_ExpoUniv1937/Expo_1937_photos.csv")

# Définir le dossier de base
chemin_base = "/home/mathieu/Documents/Stage/datasets/dataset_ExpoUniv1937/sortie/image_"

# Générer les chemins avec index formaté sur 2 chiffres
df['FICHIER'] = [f"{chemin_base}{str(i).zfill(2)}.jpg" for i in range(len(df))]

# Vérification (affiche les 5 premières lignes)
print(df.head())

# Sauvegarde dans un nouveau CSV
df.to_csv("nouveau_fichier.csv", index=False)

   Unnamed: 0                                                URL  \
0           0  https://bibliotheques-specialisees.paris.fr/ar...   
1           1  https://bibliotheques-specialisees.paris.fr/ar...   
2           2   https://gallica.bnf.fr/ark:/12148/btv1b10589915p   
3           3   https://gallica.bnf.fr/ark:/12148/btv1b105899181   
4           4   https://gallica.bnf.fr/ark:/12148/btv1b10589913s   

                                               TITRE    TYPE  \
0  [Etude publicitaire pour le] cognac Bisquit. [...  images   
1  [Etude publicitaire pour Robert Lang. Expositi...  images   
2  [Exposition des Arts et des Techniques à Paris...  images   
3  [Exposition des Arts et des Techniques à Paris...  images   
4  [Exposition des Arts et des Techniques à Paris...  images   

                                             AUTEURS  CONTRIBUTEURS EDITEURS  \
0  Vigneau, André (1892-1968) ¤ Camera (Société d...            NaN     1937   
1  Vigneau, André (1892-1968) ¤ Camera (Sociét